In [18]:
import arcgis
from arcgis.gis import GIS
from arcgis.geoenrichment import *
from arcgis.features import FeatureLayer, FeatureLayerCollection, GeoAccessor
import pandas as pd
import sys
sys.path.append('../../')
from utils import get_config, format_json
from feature_layer_utils import get_enrichment_variables, create_feature_layer
from arcgis.features.geo._array import GeoArray
from arcgis.geometry  import Geometry
import json

In [2]:
username = get_config("arcgis","username")
password = get_config("arcgis","passkey")

In [20]:
gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)
#gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=get_config("arcgis","clientid"))

In [4]:
df = pd.read_csv("../resources/full_enriched_san_diego_county_block_groups_cleaned.csv")
print(f"Shape: {df.shape}")
df.head(3)

Shape: (2057, 4241)


,objectid,state_abbr,state_fips,county_fips,stcofips,tract_fips,blockgroup_fips,fips,pop_sqmi,sqmi,...,x1130fy_a,x1130fy_i,x1002fy_x,x1002fy_a,x1002fy_i,x1003fy_x,x1003fy_a,x1003fy_i,SHAPE,totpop_cy
0,29002,CA,6,73,6073,100,1,60730001001,4788.0,0.25,...,11293.00,225.0,12946134.0,26693.06,218.0,7469031.0,15400.06,214.0,"{'rings': [[[-117.18894090906399, 32.758807764...",1199.0
1,29003,CA,6,73,6073,100,2,60730001002,5032.4,0.34,...,11843.34,236.0,19996145.0,28484.54,233.0,11682118.0,16641.19,231.0,"{'rings': [[[-117.18715990828298, 32.756859764...",1692.0
2,29004,CA,6,73,6073,201,1,60730002011,4385.0,0.20,...,7167.84,143.0,7516044.0,17120.83,140.0,4369363.0,9952.99,138.0,"{'rings': [[[-117.168410904985, 32.75683776569...",902.0


In [5]:
df['fips'] = df['fips'].astype(str)
df['fips'] = '0' + df['fips']
print(f"Shape: {df.shape}")
df.head(3)

Shape: (2057, 4241)


,objectid,state_abbr,state_fips,county_fips,stcofips,tract_fips,blockgroup_fips,fips,pop_sqmi,sqmi,...,x1130fy_a,x1130fy_i,x1002fy_x,x1002fy_a,x1002fy_i,x1003fy_x,x1003fy_a,x1003fy_i,SHAPE,totpop_cy
0,29002,CA,6,73,6073,100,1,060730001001,4788.0,0.25,...,11293.00,225.0,12946134.0,26693.06,218.0,7469031.0,15400.06,214.0,"{'rings': [[[-117.18894090906399, 32.758807764...",1199.0
1,29003,CA,6,73,6073,100,2,060730001002,5032.4,0.34,...,11843.34,236.0,19996145.0,28484.54,233.0,11682118.0,16641.19,231.0,"{'rings': [[[-117.18715990828298, 32.756859764...",1692.0
2,29004,CA,6,73,6073,201,1,060730002011,4385.0,0.20,...,7167.84,143.0,7516044.0,17120.83,140.0,4369363.0,9952.99,138.0,"{'rings': [[[-117.168410904985, 32.75683776569...",902.0


In [23]:
significant_columns_retained = ["state_abbr","state_fips","county_fips","tract_fips","blockgroup_fips", "fips", "SHAPE"]
columns_to_convert_to_string = ["state_abbr","state_fips","county_fips","tract_fips","blockgroup_fips", "fips"]

In [19]:
block_grp_df = df[significant_columns_retained]
block_grp_df["SHAPE"] = GeoArray([Geometry(json.loads(format_json(x))) for x in block_grp_df['SHAPE']])

/var/folders/jg/d8_zwblj5ql38q843xmy7h9r0000gp/T/ipykernel_39471/2852920013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  block_grp_df["SHAPE"] = GeoArray([Geometry(json.loads(format_json(x))) for x in block_grp_df['SHAPE']])


In [24]:
block_grp_df[columns_to_convert_to_string] = block_grp_df[columns_to_convert_to_string].astype(str)

/var/folders/jg/d8_zwblj5ql38q843xmy7h9r0000gp/T/ipykernel_39471/3447990820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  block_grp_df[columns_to_convert_to_string] = block_grp_df[columns_to_convert_to_string].astype(str)


In [25]:
%%time
feature_layer = block_grp_df.spatial.to_featurelayer(
                                     gis=gis,
                                     title="San Diego Block Group Boundaries",
                                     tags=['SanDiego', 'BlockGroup'],
                                    )
print(f"Number of fields published as part of this layer:",len(feature_layer.layers[0].properties['fields']))

Number of fields published as part of this layer: 9
CPU times: user 1.92 s, sys: 159 ms, total: 2.08 s
Wall time: 17.4 s
